<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Capstone: Pixel Pilot, Object Detection for Self Driving Car
# Annex Video Editing Code Snippet

> Authors: Michael King Sutanto

This Jupyter Notebook serves as an annex containing code snippets related to videos editing.  
It contains a collection of Python code snippets that demonstrate various functionalities such as cutting, combining, converting to images, and video inference.

---

### Cut Video
Cut video based on given start and end time.

In [2]:
from moviepy.editor import VideoFileClip

def cut_video(input_path, output_path, start_time, end_time, fps):
    # Load the video file
    video = VideoFileClip(input_path)
    
    # Check if video.fps is None and set fps explicitly if needed
    fps = float(fps) if fps is not None else 24.0  # Ensure fps is a float
    print("Using FPS:", fps)
    
    # Cut the clip
    clip = video.subclip(start_time, end_time)
    
    # Write the result to the output file
    clip.write_videofile(output_path, fps=fps, codec='libx264')

In [28]:
# Define your video paths and cut times
input_video_path = '../datasets/videos/test.mp4'
output_video_path = '../datasets/videos/test_short5.mp4'
start_time = (7 * 60) + 45 # converting time to seconds
end_time = (8 * 60) + 10
fps = 29.97  # Manually set this; ensure it's a float

cut_video(input_video_path, output_video_path, start_time, end_time, fps)

Using FPS: 29.97
Moviepy - Building video ../datasets/videos/test_short5.mp4.
MoviePy - Writing audio in test_short5TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ../datasets/videos/test_short5.mp4



Moviepy - Done !
Moviepy - video ready ../datasets/videos/test_short5.mp4


In [4]:
# Define your video paths and cut times
input_video_path = '../datasets/videos/test.mp4'
output_video_path = '../datasets/videos/test_short1.mp4'
start_time = (3 * 60) + 28  # converting time to seconds
end_time = (4 * 60)
fps = 60  # Manually set this; ensure it's a float

cut_video(input_video_path, output_video_path, start_time, end_time, fps)

Using FPS: 60.0
Moviepy - Building video ../datasets/videos/test_short1.mp4.
Moviepy - Writing video ../datasets/videos/test_short1.mp4



Moviepy - Done !
Moviepy - video ready ../datasets/videos/test_short1.mp4


---

### Convert Video to Images Every 15 Frames
Extract images from a video with a selected frame interval.

In [3]:
import cv2
import os

# Load the video
video_path = '../datasets/videos/train.mp4'
output_folder = '../datasets/images/'
os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)
frame_rate = 15  # the interval to save frames

frame_count = 0
saved_frame_count = 0

while True:
    success, frame = cap.read()
    if not success:
        break  # no more frames, or can't read video

    # Save frame every 15 frames
    if frame_count % frame_rate == 0:
        output_path = os.path.join(output_folder, f"frame_{saved_frame_count}.jpg")
        cv2.imwrite(output_path, frame)
        saved_frame_count += 1

    frame_count += 1

cap.release()
print(f"Extracted {saved_frame_count} images.")

Extracted 11257 images.


---

### Make Folder 1 Match Folder 2
Use case:  
For example initially we used 720p video and we already filtered out unused images. And we want to update the images to 1080p.  
We can use this code to delete unused images in the new 1080p folder to match the 720p folder.

In [ ]:
import os

# Set the paths to your folders
folder1_path = r'C:\Users\MichaelKS\GA\capstone\datasets\images'
folder2_path = r'C:\Users\MichaelKS\GA\capstone\datasets\images_720'

# List all files in both folders
files_in_folder1 = set(os.listdir(folder1_path))
files_in_folder2 = set(os.listdir(folder2_path))

# Determine which files are in Folder 1 but not in Folder 2
files_to_delete = files_in_folder1 - files_in_folder2

# Delete these files from Folder 1
for file_name in files_to_delete:
    file_path = os.path.join(folder1_path, file_name)
    os.remove(file_path)
    print(f"Deleted {file_path}")

print("Folder 1 has been updated to match Folder 2.")

---

### Video Inference

In [5]:
from ultralytics import YOLO
# Load model
model = YOLO('../model/runs/detect/train/weights/best.pt')

In [ ]:
import cv2
results = model(source='../datasets/videos/test_0.mp4', show=True, conf=0.3, save=True)

cv2.destroyAllWindows()

---

### Video Inference With Frame Rate

In [5]:
import cv2
import random
import time
import os
from ultralytics import YOLO

def get_name(file_path):
  name_idx = 0
  file_pos = (file_path).rfind('\\')

  if(file_pos == -1):
      file_pos = (file_path).rfind('/')

      if(file_pos == -1):
          file_pos = 0

  name_idx = file_pos + 1

  name = file_path[name_idx:]

  return name

def get_save_path(file_name, folder_name):
  path = "result"
  save_path = os.path.join(path, folder_name)

  exists = os.path.exists(save_path)

  if(not exists):
      os.makedirs(save_path)

  save_path = os.path.join(save_path, file_name)

  return save_path

def draw_box(img, detection_results, class_list, colors, label_size) :
  # Get information from result
  xyxy = detection_results.boxes.xyxy.numpy()
  confidence = detection_results.boxes.conf.numpy()
  class_ids = detection_results.boxes.cls.numpy().astype(int)
  # Pack together for easy use
  results = list(zip(class_ids, confidence, xyxy))
  # Copy image, in case that we need original image for something
  out_image = img.copy()

  for result in results :
    # Unpack
    class_id, con, box = result
    # Choose color
    box_color = colors[int(class_id)]
    text_color = (255,255,255)
    # Get Class Name
    label = class_list[int(class_id)]
    # Draw object box
    first_half_box = (int(box[0]),int(box[1]))
    second_half_box = (int(box[2]),int(box[3]))
    cv2.rectangle(out_image, first_half_box, second_half_box, box_color, 2)

    # Create text
    text_print = '{label} {con:.2f}'.format(label = label, con = con)
    # Locate text position
    text_location = (int(box[0]), int(box[1] - 10 ))
    # Get size and baseline
    labelSize, baseLine = cv2.getTextSize(text_print, cv2.FONT_HERSHEY_SIMPLEX, label_size, 1)

    # Draw text's background
    cv2.rectangle(out_image
                    , (int(box[0]), int(box[1] - labelSize[1] - 10 ))
                    , (int(box[0])+labelSize[0], int(box[1] + baseLine-10))
                    , box_color , cv2.FILLED)
    # Put text
    cv2.putText(out_image, text_print ,text_location
                , cv2.FONT_HERSHEY_SIMPLEX , label_size
                , text_color, 2, cv2.LINE_AA)

  return out_image

def draw_fps(avg_fps, img):
  avg_fps_str = float("{:.2f}".format(avg_fps))

  cv2.rectangle(img, (10,2), (280,50), (255,255,255), -1)
  cv2.putText(img, "FPS: "+str(avg_fps_str), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), thickness=3)

  return img

In [6]:
def detection(source, model, folder_name, img_size, label_size=1):
  # Initialize video
  cap = cv2.VideoCapture(source)

  # Initialize YOLOv8 model
  model_path = model
  yolov8_detector = YOLO(model_path)

  # Class Name and Colors
  label_map = yolov8_detector.names
  colors = [[random.randint(0, 255) for _ in range(3)] for _ in label_map]

  # FPS Detection
  frame_count = 0
  total_fps = 0
  avg_fps = 0

  # FPS Video
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  frame_width = int(cap.get(3))
  frame_height = int(cap.get(4))

  video_frames = []

  while cap.isOpened():
    # Press key q to stop
    if cv2.waitKey(1) == ord('q'):
        break

    try:
        # Read frame from the video
        ret, frame = cap.read()
        if not ret:
            break
    except Exception as e:
        print(e)
        continue

    # # Start Time
    start = time.time()
    # Update object localizer
    results = yolov8_detector.predict(frame, imgsz=img_size, verbose=False)
    result = results[0].cpu()

    # Draw Detection Results
    combined_img = draw_box(frame, result, label_map, colors, label_size)

    end = time.time()
    # # End Time

    # Draw FPS
    frame_count += 1
    fps = 1 / (end - start)
    total_fps = total_fps + fps
    avg_fps = total_fps / frame_count

    combined_img = draw_fps(avg_fps, combined_img)

    # Append frame to array
    video_frames.append(combined_img)

    #
    print("(%2d / %2d) Frames Processed" % (frame_count, total_frames))

  print("\nCreate a Video:")

  # Get a file name
  file_name = get_name(source)
  # Get Save Path
  save_path = get_save_path(file_name, folder_name)
  # Create VideoWriter object.
  out = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'XVID'), int(avg_fps), (frame_width, frame_height))

  for frame in video_frames:
    out.write(frame)

  out.release()

  print("Video is saved in: "+save_path)

In [ ]:
# Predict with FPS
detection('../datasets/videos/cam4_720_short.mp4', '../model/runs/detect/train/weights/best.pt', "predict", 736)

---

## Combine videos

In [8]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

# List of video file names
video_files = [f'../datasets/videos/predict/pred_test_{i}.avi' for i in range(0,5)]

# List to store VideoFileClip objects
clips = []

# Load each video file and store as VideoFileClip object
for file in video_files:
    clip = VideoFileClip(file)
    clips.append(clip)

# Concatenate the clips
final_clip = concatenate_videoclips(clips)

# Write the final clip to a new file
final_clip.write_videofile('../datasets/videos/predict/pred_test_combined.avi', codec='libx264', bitrate='5000k', threads=4)

Moviepy - Building video ../datasets/videos/predict/pred_test_combined.avi.
Moviepy - Writing video ../datasets/videos/predict/pred_test_combined.avi



Moviepy - Done !
Moviepy - video ready ../datasets/videos/predict/pred_test_combined.avi


In [1]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

# List of video file names
video_files = [f'../datasets/videos/predict/pred_test_{i}.avi' for i in range(1,4)]

# List to store VideoFileClip objects
clips = []

# Load each video file and store as VideoFileClip object
for file in video_files:
    clip = VideoFileClip(file)
    clips.append(clip)

# Concatenate the clips
final_clip = concatenate_videoclips(clips)

# Write the final clip to a new file
final_clip.write_videofile('../datasets/videos/predict/pred_test_combined.mp4', codec='libx264', bitrate='5000k', threads=4)

Moviepy - Building video ../datasets/videos/predict/pred_test_combined.mp4.
Moviepy - Writing video ../datasets/videos/predict/pred_test_combined.mp4



Moviepy - Done !
Moviepy - video ready ../datasets/videos/predict/pred_test_combined.mp4
